In [6]:
import pymysql.cursors
import pandas as pd
import numpy as np
import connect_to_db as cn
import json
from scipy.stats import entropy
import math
import csv
import parmap

In [7]:
# select query를 날려서 가져온 result를 DataFrame으로 반환하는 function.
def select_query_result_to_df(sql):
    connection = cn.make_connection('connection.csv')
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute(sql)
    result = cursor.fetchall()
    connection.close()
    
    result_df = pd.DataFrame(result)
    
    return result_df

In [8]:
def extract_scores(author):
    connection = cn.make_connection('connection.csv')

    cursors1 = []
    sqls1 = []
    results1 = []
    result_list1 = []
    for i in range(6):
        cursors1.append(connection.cursor(pymysql.cursors.DictCursor))

        sqls1.append(f"select count(*) from comments where author='{author}' and is_score = 1 and link_key = parent_key;")
        sqls1.append(f"select count(*) from comments where author='{author}' and is_score = 2 and link_key = parent_key;")
        sqls1.append(f"select count(*) from comments where author='{author}' and is_score = 3 and link_key = parent_key;")
        sqls1.append(f"select count(*) from comments where author='{author}' and es_score = 1 and link_key = parent_key;")
        sqls1.append(f"select count(*) from comments where author='{author}' and es_score = 2 and link_key = parent_key;")
        sqls1.append(f"select count(*) from comments where author='{author}' and es_score = 3 and link_key = parent_key;")

    for i in range(6):
        cursors1[i].execute(sqls1[i])
        # result가 dict 형태로 return.
        results1.append(cursors1[i].fetchall())

    connection.close()
    
    is_score = []
    es_score = []

    for i in range(6):
        results1[i] = pd.DataFrame(results1[i])
        if i < 3:
            is_score.extend(list(np.array(results1[i]['count(*)'])))
        else:
            es_score.extend(list(np.array(results1[i]['count(*)'])))
    
    is_score_entropy = entropy(is_score, base=3)
    es_score_entropy = entropy(es_score, base=3)
    
    # is_score_normalized_entropy = entropy(is_score) / math.log(3)
    # es_score_normalized_entropy = entropy(es_score) / math.log(3)
    
    return [author, is_score_entropy, es_score_entropy]

In [9]:
def write_entropy_csv(result_list, end_index):
    fields = ['author', 'is_score_entropy', 'es_score_entropy']
    filename = f"entropy_{end_index}.csv"
    with open(filename, 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(result_list)

In [10]:
def entropy_main(index):
    sql = f'select author from mentor where is_valid = 1 and is_score_entropy is null;'
    result_df = select_query_result_to_df(sql)
    authors = np.array(result_df['author'].astype(str).values.tolist())
    
    if index % 100000 == 0:
        start_index = index - 100000
    else:
        start_index = index - (index % 100000)
        
    result_for_csv = []

    for i in range(start_index, index):
        result_for_csv.append(extract_scores(authors[i]))
    
    write_entropy_csv(result_for_csv, index)

In [11]:
# index_list = [100000, 200000, 300000, 400000, 500000, 594997]
index_list = [100000, 200000, 300000, 400000, 466621]

if __name__ == '__main__':
    # multi processing
    parmap.map(entropy_main, index_list, pm_pbar=True, pm_processes=5)

100%|██████████| 5/5 [07:46<00:00, 93.25s/it] 
